In [1]:
#r "nuget: Newtonsoft.Json"
#r "nuget: Microsoft.Azure.Cosmos"
using Microsoft.Azure.Cosmos;
using System.Net.Http;

Installed Packages Microsoft.Azure.Cosmos, 3.53.1 Newtonsoft.Json, 13.0.4

In [2]:
var cstring = "AccountEndpoint=https://localhost:8081/;AccountKey=C2y6yDjf5/R+ob0N8A7Cgv30VRDJIWEHLM+4QDU5DE2nQ9nDuVTqobD4b8mGGyPMbIZnqyMsEcaGQy67XIw/Jw==";
var client = new CosmosClient(cstring);
var db = client.GetDatabase("StackOverflow");
var postContainer = db.GetContainer("Posts");

In [3]:
public class Post    
{
    public string id { get; set; }
    public int PostId { get; set; }
    public string PostBody { get; set; }
    public string Title { get; set; }
    public int ViewCount { get; set; }
    public int AnswerCount { get; set; }
    public int CommentCount { get; set; }
    public int FavoriteCount { get; set; }
    public int AcceptedAnswerId { get; set; }
    public DateTime? CreatedOn { get; set; }
    public DateTime? ClosedDate { get; set; }
    public int OwnerUserId { get; set; }
    public string OwnerDisplayName { get; set; }
    public string PostType { get; set; }
    public int Score { get; set; }
    public string Tags { get; set; }
}

A User adds a new question which triggers an INSERT operation.

In [4]:
var newpost = new Post
{
    id = Guid.NewGuid().ToString(),
    PostId = -10,
    PostBody = "Can anyone help me with creating a new record in a Cosmos DB container using C#?",
    Title = "How to create a new record in a Cosmos DB container?",
    ViewCount = 0,
    AnswerCount = 0,
    CommentCount = 0,
    FavoriteCount = 0,
    AcceptedAnswerId = 0,
    CreatedOn = DateTime.UtcNow,
    ClosedDate = null,
    OwnerUserId = -10,
    OwnerDisplayName = "Test User",
    PostType = "Question",
    Score = 0,
    Tags = "CosmosDB,Azure,Database"    
};

var newitem = await postContainer.CreateItemAsync(newpost, new PartitionKey(newpost.OwnerUserId));
Console.WriteLine($"Created item in database with id: {newitem.Resource.id}");
Console.WriteLine($"Request charge: {newitem.RequestCharge}");



Created item in database with id: c5e1c44b-3d3e-4729-9118-6faf7e422573
Request charge: 11.43


A User finds this question and clicks on it.

In [ ]:
var id = newitem.Resource.id;
var userid = newitem.Resource.OwnerUserId;

var postQuery = new QueryDefinition("SELECT * FROM c WHERE c.id = @id AND c.OwnerUserId = @userid")
    .WithParameter("@id", id)
    .WithParameter("@userid", userid);
    // You can change the consistency level of READ here.
    // var iterator = postContainer.GetItemQueryIterator<Post>(postQuery, requestOptions: new QueryRequestOptions { ConsistencyLevel = ConsistencyLevel.Eventual });
var iterator = postContainer.GetItemQueryIterator<Post>(postQuery);
var results = new List<Post>();
while (iterator.HasMoreResults)
{
    var response = await iterator.ReadNextAsync();
    results.AddRange(response);
    Console.WriteLine($"Request charge: {response.RequestCharge}");
}
results[0].Display();

Request charge: 2.92


id,c5e1c44b-3d3e-4729-9118-6faf7e422573
PostId,-10
PostBody,Can anyone help me with creating a new record in a Cosmos DB container using C#?
Title,How to create a new record in a Cosmos DB container?
ViewCount,0
AnswerCount,0
CommentCount,0
FavoriteCount,0
AcceptedAnswerId,0
CreatedOn,2025-09-18 18:24:59Z
ClosedDate,<null>


If you can make id and partitionid available, you can make this operation much more affordable and fast by using READ operation rather than QUERY

In [9]:
var post = await postContainer.ReadItemAsync<Post>(id, new PartitionKey(userid));
Console.WriteLine($"Request charge: {post.RequestCharge}");
post.Resource.Display();

Request charge: 1


id,34b41e20-6d93-4a80-93cb-79614a4443e1
PostId,-10
PostBody,Can anyone help me with creating a new record in a Cosmos DB container using C#?
Title,How to create a new record in a Cosmos DB container?
ViewCount,0
AnswerCount,0
CommentCount,0
FavoriteCount,0
AcceptedAnswerId,0
CreatedOn,2025-05-14 19:09:44Z
ClosedDate,<null>


A User answers the question which triggers an Update Operation.

In [10]:
var query = new QueryDefinition("SELECT * FROM c WHERE c.PostId = @postId")
    .WithParameter("@postId", 10);
var iterator = postContainer.GetItemQueryIterator<Post>(query);
var results = new List<Post>();
while (iterator.HasMoreResults)
{
    var response = await iterator.ReadNextAsync();
    results.AddRange(response);
}
results[0].Display();
results[0].ViewCount += 1;
results[0].AnswerCount += 1;
results[0].Score += 10;
//Figure out the partition key
var updateResponse = await postContainer.ReplaceItemAsync(results[0], results[0].id, new PartitionKey(results[0].OwnerUserId));
updateResponse.Resource.Display();
Console.WriteLine($"Request charge: {updateResponse.RequestCharge}");

Error: System.ArgumentOutOfRangeException: Index was out of range. Must be non-negative and less than the size of the collection. (Parameter 'index')
   at System.Collections.Generic.List`1.get_Item(Int32 index)
   at Submission#10.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

We can use Partial Update to make this code and request package lighter.
Partial Update request contains only the properties needs to get updated.

In [ ]:
var _id = "20";
var pkey = new PartitionKey("key");

var ops = new List<PatchOperation>();
ops.Add(PatchOperation.Increment("/ViewCount", 1));
ops.Add(PatchOperation.Increment("/AnswerCount", 1));
ops.Add(PatchOperation.Increment("/Score", 10));

var result = await container.PatchItemAsync<Sale>(
    id: _id, 
    partitionKey: pkey, 
    patchOperations: ops);

Console.WriteLine($"Request charge: {result.RequestCharge}");
result.Resource.Display();


Another way to query data is the STREAM API
Partition Key is required.
There will be no serialization which will make the operation lighter.

In [ ]:
var query = new QueryDefinition("SELECT * FROM c WHERE c.PostId = @postId")
    .WithParameter("@postId", -1);
var iterator = postContainer.GetItemQueryStreamIterator(query,null, new QueryRequestOptions { PartitionKey = new PartitionKey(results[0].id) });
while (queryResultSetIterator.HasMoreResults)
{
    using (ResponseMessage response = await queryResultSetIterator.ReadNextAsync())
    {
        using (StreamReader sr = new StreamReader(response.Content))
        using (JsonTextReader jtr = new JsonTextReader(sr))
        {
            JObject result = JObject.Load(jtr);
            result.Display();
        }
    }
}

System finds this question a duplicate and deletes the question

In [ ]:
var id = results[0].id;
var pkey = results[0].ownerUserId.ToString();
var deleteResponse = postContainer.DeleteItemAsync<Post>(id, new PartitionKey(pkey));
Console.WriteLine($"Delete Operation Cost {deleteResponse.Result.RequestCharge} RU/s");